# Goal of this notebook:

- Create a visualization for Ball and Gaze on Screen for one trial

### Some Acronyms:
#### WCS : World Coordinate System
#### HCS : Head Coordinate System
#### SCS : Screen Coordinate System

### Import Packages

In [1]:
from __future__ import division

import PerformParser as pp
import pandas as pd
import numpy as np
from scipy import signal as sig
import performFun as pF

#import bokeh.plotting as bkP
#import bokeh.models as bkM
#from bokeh.palettes import Spectral6
#bkP.output_notebook() 

import cv2
import os
import scipy.io as sio
import matplotlib

#%matplotlib notebook
#from ipywidgets import interact
#import filterpy as fP
#from bokeh.io import push_notebook

import Quaternion as qu
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz
from scipy.fftpack import fft
from mpl_toolkits.mplot3d import Axes3D

#bkP.output_notebook()

#bkP.output_file('timeSeries.html') 

#%pylab inline
#%matplotlib notebook


### Importing Experiment Data and Creating "Raw", "Calibration" and "Processed" Data Frames

In [2]:


# List of subjects with good calibration quality
fileTimeList = ['2016-4-19-14-4', '2016-4-22-11-57', '2016-4-27-13-28', '2016-4-28-10-57', '2016-4-29-11-56',
                '2016-5-3-12-52', '2016-5-4-13-3', '2016-5-5-13-7', '2016-5-6-11-2', '2016-5-6-13-4']

fileTime = '2018-4-25-17-48'
fileTime = '2018-6-13-16-27'
fileTime = '2018-7-17-18-59'
fileTime = '2018-7-18-18-53'
fileTime = '2018-8-6-19-24'
fileTime = '2018-8-9-11-15'
fileTime = '2018-8-9-21-46'
fileTime = '2018-8-14-10-49'
fileTime = '2018-8-14-12-20'
fileTime = '2018-8-16-16-35'
fileTime = '2018-8-17-13-24'
fileTime = '2018-8-22-16-31'
fileTime = '2018-8-28-12-22'
fileTime = '2018-8-29-12-11'
fileTime = '2018-8-30-9-34'
fileTime = '2018-8-30-12-5'
fileTime = '2018-8-30-13-42'
fileTime = '2018-8-30-15-1'
fileTime = '2018-8-30-15-57'
fileTime = '2018-8-31-13-32'
fileTime = '2018-8-31-15-14'
fileTime = '2018-8-31-16-42'
fileTime = '2018-9-1-11-27'
fileTime = '2018-9-4-10-58'
fileTime = '2018-9-4-15-12'
fileTime = '2018-9-5-15-4'
fileTime = '2018-9-5-16-39'
fileTime = '2018-9-6-12-15'
fileTime = '2018-9-6-13-51'
fileTime = '2018-9-7-12-10'
#fileTime = '2016-12-2-19-22' # Evan's Data
#fileTime = '2016-4-19-14-4'
#fileTime = '2016-4-22-11-57'
#fileTime = '2016-4-27-13-28' # Moderate Gaze Errors
#fileTime = '2016-4-28-10-57' # Moderate Gaze Errors
#fileTime = '2016-4-29-11-56' # Good Gaze Errors
fileTime = '2016-5-3-12-52' # Very clean data Low gaze errors
#fileTime = '2016-5-4-13-3' # Good Gaze Errors
#fileTime = '2016-5-5-13-7'
#fileTime = '2016-5-6-11-2' # High Gaze Errors
#fileTime = '2016-5-6-13-4' # Good Gaze Errors

expCfgName = "gd_pilot.cfg"
sysCfgName = "PERFORMLAB1.cfg"

filePath = "../Data/" + fileTime + "/"
fileName = "exp_data-" + fileTime

sessionDict = pF.loadSessionDict(filePath,fileName,expCfgName,sysCfgName,startFresh=False)

rawDataFrame = sessionDict['raw']
processedDataFrame = sessionDict['processed']
calibDataFrame = sessionDict['calibration']
trialInfoDataFrame = sessionDict['trialInfo']

SecDataFrame Called!
Creating Secondary Data Frame
1
DataFrame
DataFrame OK!
2
['IOD', 'IPD', 'ballFinalPos_XYZ', 'ballInitialPos_XYZ', 'ballInitialVel_XYZ', 'ballPos_XYZ', 'ballTTC', 'ballVel_XYZ', 'blankDur', 'blockNumber', 'calibrationCounter', 'calibrationPos_XYZ', 'cycEyeBasePoint_XYZ', 'cycEyeInHead_XYZ', 'cycEyeNodeInWorld_XYZ', 'cycEyeOnScreen_XY', 'cycGazeNodeInWorld_XYZ', 'cycInverseMat_4x4', 'cycMat_4x4', 'eventFlag', 'eyeTimeStamp', 'frameTime', 'inCalibrationQ', 'isBallVisibleQ', 'isCalibratedSMIQ', 'leftEyeBasePoint_XYZ', 'leftEyeInHead_XYZ', 'leftEyeInverseMat_4x4', 'leftEyeLensDistance', 'leftEyeMat_4x4', 'leftEyeNodeInHead_XYZ', 'leftEyeNodeInWorld_XYZ', 'leftEyeOnScreen_XY', 'leftEyeScreenDistance', 'leftGazeNodeInWorld_XYZ', 'leftPupilPos_XYZ', 'leftPupilRadius', 'paddleMat_4x4', 'paddlePos_XYZ', 'paddleQuat_XYZW', 'postBlankDur', 'preBlankDur', 'rightEyeBasePoint_XYZ', 'rightEyeInHead_XYZ', 'rightEyeInverseMat_4x4', 'rightEyeLensDistance', 'rightEyeMat_4x4', 'rightE

In [3]:
#rawDataFrame.loc[:,[('frameTime',''),('eventFlag','')]]

In [6]:
calibDataFrame.head(5)

IOD      IPD ballFinalPos       ballInitialPos       ballInitialVel     \
                               X  Y  Z              X  Y  Z              X  Y   
0  60.3639  61.8813            0  0  0              0  0  0              0  0   
1  60.3639  61.8813            0  0  0              0  0  0              0  0   
2  60.3656  61.9375            0  0  0              0  0  0              0  0   
3  60.3677  61.8514            0  0  0              0  0  0              0  0   
4  60.3703  61.8784            0  0  0              0  0  0              0  0   

     ballPos           ballTTC ballVel           blankDur blockNumber      
   Z       X    Y    Z               X    Y    Z                           
0  0     NaN  NaN  NaN     1.6     NaN  NaN  NaN      0.5           0 ...  
1  0     NaN  NaN  NaN     1.6     NaN  NaN  NaN      0.5           0 ...  
2  0     NaN  NaN  NaN     1.6     NaN  NaN  NaN      0.5           0 ...  
3  0     NaN  NaN  NaN     1.6     NaN  NaN  NaN      0.5           0 ...  
4  0     NaN  NaN  NaN     1.6     NaN  NaN  NaN      0.5           0 ...  

[5 rows x 236 columns]

In [5]:
processedDataFrame.head(5)

,frameTime,eventFlag,trialNumber
,,,
0,632.4261,trialStart,0
1,632.4395,False,0
2,632.4527,False,0
3,632.4661,False,0
4,632.4794,False,0


In [6]:
sessionDict = pF.rawDataProcessing(sessionDict)


Low Pass/Median Filtering Gaze Data...
... Done!

Calculating Cyc Gaze Velocity...
... Done!

Calculating True POR on Screen...
('Size of Calibration Data Frame is', 5900)
('Size of TruePOR array:', (5900, 3))


/Users/kamranbinaee/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/pandas/core/indexing.py:344: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
  self.obj[item] = s



Calculating True POR on Screen...

Calculating Calibration Homography...
('Frame Range =[', 0, ' ', 2699, ']')
(2700, 2)
('MSE_after = ', [2022.6865882563034])
('MSE_before = ', [6914.9999150921385])
(1000, 1026)
('H = ', array([[  9.03961675e-01,  -5.95915186e-02,   1.06596999e-04],
       [  3.27034596e-02,   1.88746781e+00,   2.73963745e-03],
       [  2.22822889e+00,   7.12301694e+00,   1.00000000e+00]]))
... Done!

Calculating Gaze & Ball On Screen...
... Done!

Calculating Gaze Error EIH ...
... Done!

Calculating Gaze Error GIW ...
... Done!

Calculating Ball and Head Velocity ...
.... Done!

Calculating Event Indexes ...
... Done!


In [7]:
print (list(sessionDict['trialInfo'].columns))
(sessionDict['processed'], sessionDict['trialInfo']) = pF.calculateCrossingFrame(sessionDict['raw'], sessionDict['processed'], sessionDict['trialInfo'])
rawDataFrame = sessionDict['raw']
processedDataFrame = sessionDict['processed']
calibDataFrame = sessionDict['calibration']
trialInfoDataFrame = sessionDict['trialInfo']

rawDataFrame.loc[:,('SubjectID', '')] = fileTime
processedDataFrame.loc[:,('SubjectID', '')] = fileTime
calibDataFrame.loc[:,('SubjectID', '')] = fileTime


['ballCaughtFr', 'ballCaughtQ', 'blankDur', 'postBlankDur', 'preBlankDur', 'firstFrame', 'lastFrame', 'trialStartIdx', 'ballOffIdx', 'ballOnIdx', 'ballCrossingIndex']


In [8]:
#list(processedDataFrame.columns)
trialInfoDataFrame.head(5)

,ballCaughtFr,ballCaughtQ,blankDur,postBlankDur,preBlankDur,firstFrame,lastFrame,trialStartIdx,ballOffIdx,ballOnIdx,ballCrossingIndex
trialNum,,,,,,,,,,,
0,63,False,0.5,0.1,0.3,0,164,0,23,61,63
1,267,False,0.5,0.3,0.6,165,329,165,246,264,267
2,431,False,0.5,0.3,0.6,330,494,330,377,406,431
3,559,False,0.5,0.1,0.3,495,659,495,528,544,559
4,762,False,0.5,0.3,0.6,660,824,660,696,736,762


In [ ]:

sessionDict['raw'] = rawDataFrame
sessionDict['processed'] = processedDataFrame
sessionDict['calibration'] = calibDataFrame
sessionDict['trialInfo'] = trialInfoDataFrame
pd.to_pickle(sessionDict, filePath + fileName + '.pickle')
print ('...Done! Done!\n')


### Here I'm Filtering the raw CycOnScreen data using
#### - "Windowed Median" and Windowed Average Filters are used
#### - It seems that windowed median with window size = 3 is necessary for removing outliers due to bad tracking frames
#### - So the Averaging Filter is applied on median filtered data 

In [ ]:
'''
# Creating an Empty Data Frame to store the filtered Data and attach it to the processed Data Frame
cyc_X = rawDataFrame['cycEyeOnScreen']['X'].values
cyc_Y = rawDataFrame['cycEyeOnScreen']['Y'].values

medianCyc_X = pd.rolling_median(cyc_X, 3, min_periods = 0)
medianCyc_Y = pd.rolling_median(cyc_Y, 3, min_periods = 0)

processedDataFrame.loc[:,('medFilt3_cycEyeOnScreen', 'X')] = pd.rolling_median(cyc_X, 3, min_periods = 0)
processedDataFrame.loc[:,('medFilt3_cycEyeOnScreen', 'Y')] = pd.rolling_median(cyc_Y, 3, min_periods = 0)
processedDataFrame.loc[:,('medFilt5_cycEyeOnScreen', 'X')] = pd.rolling_median(cyc_X, 5, min_periods = 0)
processedDataFrame.loc[:,('medFilt5_cycEyeOnScreen', 'Y')] = pd.rolling_median(cyc_Y, 5, min_periods = 0)
processedDataFrame.loc[:,('medFilt7_cycEyeOnScreen', 'X')] = pd.rolling_median(cyc_X, 7, min_periods = 0)
processedDataFrame.loc[:,('medFilt7_cycEyeOnScreen', 'Y')] = pd.rolling_median(cyc_Y, 7, min_periods = 0)

processedDataFrame.loc[:,('avgFilt3_cycEyeOnScreen', 'X')] = pd.rolling_mean(medianCyc_X, 3, min_periods = 0)
processedDataFrame.loc[:,('avgFilt3_cycEyeOnScreen', 'Y')] = pd.rolling_mean(medianCyc_Y, 3, min_periods = 0)
processedDataFrame.loc[:,('avgFilt5_cycEyeOnScreen', 'X')] = pd.rolling_mean(medianCyc_X, 5, min_periods = 0)
processedDataFrame.loc[:,('avgFilt5_cycEyeOnScreen', 'Y')] = pd.rolling_mean(medianCyc_Y, 5, min_periods = 0)

#pF.plotFilteredGazeData(rawDataFrame, processedDataFrame)
list(processedDataFrame.columns)
'''

In [ ]:
list(rawDataFrame.columns)
#processedDataFrame
#print(calibDataFrame.trialNumber.unique())
#gbCalibSession = calibDataFrame.groupby(['trialNumber'])
#a = np.append(gbCalibSession.get_group(1103).index, gbCalibSession.get_group(1108).index)
#print(calibDataFrame.trialNumber.unique())
#b = np.append(gbCalibSession.get_group(1110).index, gbCalibSession.get_group(1116).index)
#c = np.append(a,b)
#d = np.append(gbCalibSession.get_group(1212).index, gbCalibSession.get_group(1217).index)
#print(np.append(c,d))

In [ ]:
# Creating an Empty Data Frame to store the Gaze Velocity and attach it to the processed Data Frame

#processedDataFrame.loc[:, ('cycGazeVelocity', '')] = pF.calculateCycGazeVelocity(processedDataFrame,
                                                                                    trialID = None, plottingFlag = False)
#print len(processedDataFrame.cycGazeVelocity.values)
#processedDataFrame

## Group by trialNumber / inCalibrationQ 

In [ ]:
gbCalibrationQ = calibDataFrame.groupby(['inCalibrationQ'])
gbTrial = rawDataFrame.groupby(['trialNumber'])
processedGBTrial = processedDataFrame.groupby(['trialNumber'])

In [ ]:
#calibDataFrame

## The True POR points are calculated here
#### - True POR is the Position of the Calibration Spheres on the HMD Screen in SCS
#### - Remember that During Calibration procedure, Calibration Spheres are positioned in HCS (No need to take into account head Rotation)
#### - Ray Tracing method is used to find the intersection of  eye-sphere line and the HMD screen (plane)
#### - This procedure could be improved because for all 100 frames the calibration sphere is always the same!

In [ ]:
# Attaching the calculated Values to the CalibDataFrame
truePOR = pF.find2DTruePOR(calibDataFrame)
calibDataFrame.loc[:, ('cycTruePOR', 'X')] = truePOR[:,0]
calibDataFrame.loc[:, ('cycTruePOR', 'Y')] = truePOR[:,1]
calibDataFrame.loc[:, ('cycTruePOR', 'Z')] = truePOR[:,2]
print 'Size of Calibration Data Frame: ', len(calibDataFrame)
#calibDataFrame

In [ ]:
#calibDataFrame

## Calculating the Calibration Matrix (Homography) and plotting the results

In [ ]:
H = pF.calculateLinearHomography(calibDataFrame, plottingFlag = True)
print 'H = ', H
tempMatrix = np.zeros((len(processedDataFrame),3,3))
tempMatrix[:,0:3,0:3] = H
tempMatrix = tempMatrix.reshape((len(processedDataFrame),9))
#print 'H =',H
#print 'M =', tempMatrix

processedDataFrame.loc[:, ('linearHomography','0')] = tempMatrix[:,0]
processedDataFrame.loc[:, ('linearHomography','1')] = tempMatrix[:,1]
processedDataFrame.loc[:, ('linearHomography','2')] = tempMatrix[:,2]
processedDataFrame.loc[:, ('linearHomography','3')] = tempMatrix[:,3]
processedDataFrame.loc[:, ('linearHomography','4')] = tempMatrix[:,4]
processedDataFrame.loc[:, ('linearHomography','5')] = tempMatrix[:,5]
processedDataFrame.loc[:, ('linearHomography','6')] = tempMatrix[:,6]
processedDataFrame.loc[:, ('linearHomography','7')] = tempMatrix[:,7]
processedDataFrame.loc[:, ('linearHomography','8')] = tempMatrix[:,8]

#### How to access one Homography

In [ ]:
#processedDataFrame['linearHomography'].values[0].reshape((3,3))

# Calculating the Ball and Gaze Position on the HMD Screen

In [ ]:
trialID = None
#trialID = 53 # Failed Trial

[gazePoint_fr_XYZ, rotatedGazePoint_fr_XYZ,
 ballOnScreen_fr_XYZ, rotatedBallOnScreen_fr_XYZ] = pF.calculateGazeBallOnScreen(rawDataFrame, processedDataFrame, trialID)

In [ ]:

processedDataFrame.loc[:, ('gazePoint', 'X')] = gazePoint_fr_XYZ[:,0]
processedDataFrame.loc[:, ('gazePoint', 'Y')] = gazePoint_fr_XYZ[:,1]
processedDataFrame.loc[:, ('gazePoint', 'Z')] = gazePoint_fr_XYZ[:,2]

processedDataFrame.loc[:, ('rotatedGazePoint', 'X')] = rotatedGazePoint_fr_XYZ[:,0]
processedDataFrame.loc[:, ('rotatedGazePoint', 'Y')] = rotatedGazePoint_fr_XYZ[:,1]
processedDataFrame.loc[:, ('rotatedGazePoint', 'Z')] = rotatedGazePoint_fr_XYZ[:,2]

processedDataFrame.loc[:, ('ballOnScreen', 'X')] = ballOnScreen_fr_XYZ[:,0]
processedDataFrame.loc[:, ('ballOnScreen', 'Y')] = ballOnScreen_fr_XYZ[:,1]
processedDataFrame.loc[:, ('ballOnScreen', 'Z')] = ballOnScreen_fr_XYZ[:,2]

processedDataFrame.loc[:, ('rotatedBallOnScreen', 'X')] = rotatedBallOnScreen_fr_XYZ[:,0]
processedDataFrame.loc[:, ('rotatedBallOnScreen', 'Y')] = rotatedBallOnScreen_fr_XYZ[:,1]
processedDataFrame.loc[:, ('rotatedBallOnScreen', 'Z')] = rotatedBallOnScreen_fr_XYZ[:,2]
#processedDataFrame


In [ ]:
gazeX = processedDataFrame.gazePoint.X.values
gazeY = processedDataFrame.gazePoint.Y.values
gazeZ = processedDataFrame.gazePoint.Z.values

ballX = processedDataFrame.ballOnScreen.X.values
ballY = processedDataFrame.ballOnScreen.Y.values
ballZ = processedDataFrame.ballOnScreen.Z.values

gazeVector = np.array([gazeX, gazeY, gazeZ], dtype = float).T
ballVector = np.array([ballX, ballY, ballZ], dtype = float).T
#print gazeVector.shape
angle = []
for gV, bV in zip(gazeVector, ballVector):
    angle.append(pF.vectorAngle(gV,bV))
#len(angle)
processedDataFrame.loc[:,('gazeError_HCS', '')] = angle

In [ ]:

angleList = []
for i in range(len(processedDataFrame)):
    temp = pF.vectorAngle(processedDataFrame.rotatedGazePoint.values[i], processedDataFrame.rotatedBallOnScreen.values[i])
    angleList.append(temp)

eyeToScreenDistance = 0.0725
gazeError = np.array(processedDataFrame.rotatedGazePoint.values - processedDataFrame.rotatedBallOnScreen.values)
x = gazeError[:,0]
y = gazeError[:,1]
z = gazeError[:,2]

x = np.array(x,dtype = float)
y = np.array(y,dtype = float)
z = np.array(z,dtype = float)

processedDataFrame.loc[:,('gazeError_WCS','X')] = (180/np.pi)*np.arctan(x/eyeToScreenDistance)
processedDataFrame.loc[:,('gazeError_WCS','Y')] = (180/np.pi)*np.arctan(y/eyeToScreenDistance)
processedDataFrame.loc[:,('gazeError_WCS','Z')] = (180/np.pi)*np.arctan(z/eyeToScreenDistance)
processedDataFrame.loc[:,('gazeAngularError', '')] = np.array(angleList, dtype = float)


In [ ]:
#processedDataFrame

In [ ]:
#rawDataFrame = sessionDict['raw']
sessionDict['processed'] = processedDataFrame
sessionDict['calibration'] = calibDataFrame
pd.to_pickle(sessionDict, filePath + fileName + '.pickle')

In [ ]:
processedDataFrame.columns

In [ ]:
rawDataFrame